In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#cell_A = np.array([1,0,0,1,1,0,1,0,0,1,1,0,0,1,0,0,1,0,1,0])
#cell_B = np.array([1,0,0,0,1,0,0,1,0,1,0,1,1,0,0,1,1,0,0,0])
#cell_C = np.array([1,0,0,0,0,1,0,0,1,0,1,0,1,1,0,0,0,1,0,1])
#cell_D = np.array([0,1,1,0,0,1,0,0,0,0,1,0,0,0,1,0,1,1,0,1])
#cell_E = np.array([1,0,0,1,0,0,1,0,1,1,0,1,1,1,0,1,0,0,1,0])
#cell_F = np.array([1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0])
#cell_G = np.array([1,0,0,1,0,0,0,0,1,1,0,1,0,1,0,1,0,0,1,0])

cell_A = np.array([0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1])
cell_B = np.array([0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1])
cell_C = np.array([0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0])
cell_D = np.array([1,0,1,0,1,0,0,0,1,0,1,0,1,1,1,0,1,0,1,0,1,0,1,0])
cell_E = np.array([1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0])
cell_F = np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
cell_G = np.array([0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1])


spiketrain_arr = np.vstack( [ cell_A , cell_B , cell_C , cell_D , cell_E , cell_F , cell_G ])
cell_id_list = [0,1,2,3,4,5,6]
print(spiketrain_arr.shape)
print(cell_id_list)

(7, 24)
[0, 1, 2, 3, 4, 5, 6]


In [3]:
## input: list of probabilities 
## returns: entropy
def calc_entr(prob_arr):
    inf_list=[]
    for p in prob_arr:
        if p == 0:
            p = 1e-19
        inf_list.append( -p*np.log(p) )
    return np.sum(inf_list)

## input: list containing either a single, or 2 spike trains
## returns: probability of having a spike in time bin 
def calc_prob(spike_trains):
    if len(spike_trains)>1:
        sync_train = np.bitwise_and( spike_trains[0] , spike_trains[1] )
        tot_spikes = np.sum(sync_train)
        num_bins = sync_train.shape[0]
 
    else:
        tot_spikes = np.sum(spike_trains[0])
        num_bins = spike_trains[0].shape[0]
        
    if tot_spikes == 0:
            tot_spikes = 1e-19
    return tot_spikes / num_bins

## input: array of cell responses [rowxcol = cellsxtime bins]
## returns: 2 arrays, one containing entropy change from combining pair of spike trains
## for each pair in the input array. The other contains the indices of the combinations
def calc_entr_change(cell_array):
    num_cells = cell_array.shape[0]
    comb_list = []
    delH_list = []
    for i, train_1 in enumerate(cell_array):
        for j in range( i+1 , num_cells ):
            train_2 = cell_array[ j , : ]
            p1 = calc_prob( [train_1] )
            p2 = calc_prob( [train_2] )
            p12 = calc_prob( [train_1 , train_2] )
            p1_2 = p1 - p12
            p12_ = p2 - p12
            E_pre = calc_entr( [p1 , 1-p1 , p2 , 1-p2])
            E_post = calc_entr( [p12 , 1-p12 , p1_2 , 1-p1_2 , p12_ , 1-p12_])
            delH_list.append(E_pre - E_post)
            comb_list.append([i,j])
    return np.array(delH_list) , np.array(comb_list)


In [4]:
tolerance = 0.4
counter=0

#initialize delH_arr and comb_arr
delH_arr , comb_arr = calc_entr_change(spiketrain_arr)
while delH_arr.max() > tolerance:
    counter+=1
#figure out which pair of spike trains in spiketrain array give highest change in
#H when replaced by a synthesized train of their union and the residual trains left over
    delH_arr , comb_arr = calc_entr_change(spiketrain_arr)
    idx = np.where( delH_arr == delH_arr.max() )[0][0]
    best_comb = comb_arr[idx]
    print(best_comb)
#generate the new synthesized spike train from the best combination, and
#also generate residual trains of left over spikes
    new_train = np.bitwise_and( spiketrain_arr[ best_comb[0] ] , 
                                spiketrain_arr[ best_comb[1] ] )
    residual_train_1 = np.bitwise_xor( spiketrain_arr[ best_comb[0] ] , 
                                    new_train )
    residual_train_2 = np.bitwise_xor( spiketrain_arr[ best_comb[1] ] , 
                                    new_train )
#remove the best combination trains and add the synthesized and residual 
#trains to the spiketrain array    
    spiketrain_arr = np.delete( spiketrain_arr , best_comb , axis=0 )
    spiketrain_arr = np.vstack( [spiketrain_arr , new_train , 
                                 residual_train_1 , residual_train_2] )
#add the ids for the synthesized and residual spikes to cell identity array
#have to add the new trains before deleting the old ones because otherwise
#the idcs being referenced in best_comb will no longer be accurate
    id1 = str(cell_id_list[ best_comb[0] ])
    id2 = str(cell_id_list[ best_comb[1] ])
    new_id1 = '(' + id1 + ' ' + id2 + ')'
    new_id2 = '(-' + id1 + ' ' + id2 + ')'
    new_id3 = '(' + id1 + ' ' + '-' + id2 + ')'
    cell_id_list.append( new_id1 )
    cell_id_list.append( new_id2 )
    cell_id_list.append( new_id3 )
#remove the id of the best combination trains, starting from largest idx first
#so as not to reorder the idcs after each delete
    for ele in sorted(best_comb, reverse = True):
        del cell_id_list[ele]

    #print(spiketrain_arr)
    print(delH_arr.max())
else:
    print('finished after '+str(counter)+' iterations')
    print(spiketrain_arr)
    print(cell_id_list)

[0 1]
0.6931471805599453
[4 5]
0.6931471805599453
[1 2]
0.35021301926450943
finished after 3 iterations
[[0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 0 1 0 0 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
[2, 5, '(-0 1)', '(0 -1)', '(6 (0 1))', '(-6 (0 1))', '(6 -(0 1))', '(3 4)', '(-3 4)', '(3 -4)']
